# Welcome to the Point Clouds Workshop
In this workshop, we're going to combine 3D Machine Learning fonctions to build a pipeline for Clustering(object detection), RANSAC(segmentation), and lane line estimation.

## Data & Imports

In [ ]:
import open3d as o3d
import glob
import numpy as np
import matplotlib.pyplot as plt
from open3d.visualization.draw_plotly import get_plotly_fig

## Visualization

In [ ]:
point_cloud_files = sorted(glob.glob("data/KITTI_PCD/*.pcd"))
idx = 357
print(point_cloud_files[idx])   
point_cloud =  o3d.io.read_point_cloud(point_cloud_files[idx])


In [ ]:
points = np.asarray(point_cloud.points)
colors = np.asarray(point_cloud.colors)

In [ ]:
print(points)

In [ ]:
print(colors)

In [ ]:
import plotly.graph_objects as go

distances = np.linalg.norm(points, axis=1)

fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=distances,  # use distances for color
        colorscale='Viridis',  # choose a colorscale
        colorbar=dict(title="Distance from Origin"),  # add a colorbar title
        opacity=0.8
    )
)])
fig.update_scenes(aspectmode='data')

fig.show()

**Dark Mode?**

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=distances,  # use distances for color
        colorscale='Inferno',  # choose a colorscale Dark Mode:Inferno
        colorbar=dict(title="Distance from Origin", bgcolor="white"),  # add a colorbar title
        opacity=0.8
    )
)])

fig.update_layout(
    scene=dict(
        xaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        aspectmode='data',
        camera=dict(
            up=dict(x=-0.2, y=0, z=1),
            center=dict(x=0.2, y=0, z=0.2),
            eye=dict(x=-0.5, y=0, z=0.2))
    ),
    plot_bgcolor='black',
    paper_bgcolor='black',
    scene_dragmode='orbit'
)
fig.show()

**Reflectance**<p>
You've seen reflectance before. In this workshop, we're going to go much further in this idea, and actually USE IT.

In [ ]:
def vis_pcd(point_cloud, save="False", show=True):
    fig = get_plotly_fig(point_cloud, width = 800, height = 533, mesh_show_wireframe =False,
                            point_sample_factor = 1, front = (1,1,1), lookat =(1,1,1), up=(1,1,1), zoom=1.0)
    #fig.update_scenes(aspectmode='data')
    fig.update_layout(
    scene=dict(
        #xaxis=dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title=''),
        xaxis=dict(visible=False,range=[-70,70]),
        yaxis=dict(visible=False,range=[-40,40]),
        zaxis=dict(visible=False,range = [-5,1]),
        aspectmode='manual', aspectratio= dict(x=2, y=1, z=0.1),
        camera=dict(
            #up=dict(x=-0.2, y=0, z=0.3),
            up = dict(x=0.15, y =0, z=1),
            center=dict(x=0, y=0, z=0.1),
            #eye=dict(x=-0.5, y=0, z=0.2)
            eye = dict(x = -0.3, y=0, z=0.2)
        )
    ),
    plot_bgcolor='black',
    paper_bgcolor='black',
    scene_dragmode='orbit'
)
    if show == True:
        fig.show()

    if save != "False":
        fig.write_image("output/"+save+"_processed.jpg", scale=3)

    return fig

point_cloud = o3d.io.read_point_cloud(point_cloud_files[idx])
fig = vis_pcd([point_cloud])

## How to use Reflectance: Lane Line Detection
Because it can detect all these cool things, we can use a LiDAR to detect lane lines in 3D directly. We can use it to detect license plates, traffic signs, and, of course, cars. **Reflectance can be used as a filter**, a threshold, and we could also fuse this with camera outputs to get the best out of it.

two techniques to find the lane lines:
* Thresholding
* ROI
They are simple, yet effective (for now).<p>

#### **Thresholding**
Let's begin with the idea of a threshold, and only visualize the high reflectance points.

In [ ]:
def reflectivity_threshold(pcd, thresh=0.5):
    colors = np.asarray(pcd.colors)
    reflectivities = colors[:, 0]
    # Get the point coordinates
    points = np.asarray(pcd.points)
    # Create a mask of points that have reflectivity above the threshold
    mask = reflectivities > thresh

    # Filter points and reflectivities using the mask
    filtered_points = points[mask]

    # Create a new point cloud with the filtered points
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)
    return filtered_point_cloud

#point_cloud = o3d.io.read_point_cloud(point_cloud_files[idx])
filtered_point_cloud = reflectivity_threshold(point_cloud, thresh=0.5)

In [ ]:
fig = vis_pcd([point_cloud.paint_uniform_color((0.2,0.2,0.2)),filtered_point_cloud])

#### **Region of Interest**

In [ ]:
def roi_filter(pcd, roi_min=(0,-3,-2), roi_max=(20,3,0)):
    points = np.asarray(pcd.points)
    mask_roi = np.logical_and.reduce((
        points[:, 0] >= roi_min[0],
        points[:, 0] <= roi_max[0],
        points[:, 1] >= roi_min[1],
        points[:, 1] <= roi_max[1],
        points[:, 2] >= roi_min[2],
        points[:, 2] <= roi_max[2]
    ))
    new_filtered_points=points[mask_roi]
    # Create a new point cloud with the filtered points
    roi_pcd = o3d.geometry.PointCloud()
    roi_pcd.points = o3d.utility.Vector3dVector(new_filtered_points)
    return roi_pcd
roi_pcd = roi_filter(filtered_point_cloud)
fig = vis_pcd([point_cloud.paint_uniform_color((0.2,0.2,0.2)),roi_pcd])

One of the key advantages is that the selected values depend on the car itself. This means that regardless of the situation, we'll always have the appropriate reflectance thresholding readily available. On the flip side, the downside is that it's a highly manual process.<p>

### **Lane Line Detection Pipeline**

In [ ]:
def lane_line_pipeline(pcd):
    filtered_point_cloud=reflectivity_threshold(pcd, thresh=0.5)
    roi_pcd=roi_filter(filtered_point_cloud, roi_min=(0,-3,-2), roi_max=(20,3,0))
    return roi_pcd, filtered_point_cloud, pcd

point_cloud = o3d.io.read_point_cloud(point_cloud_files[idx])
roi_pcd, filtered_point_cloud, pcd = lane_line_pipeline(point_cloud)

fig = vis_pcd([pcd.paint_uniform_color((0.2,0.2,0.2)),roi_pcd], show=True)

**Video**

In [ ]:
import cv2
from tqdm import tqdm

output_handle = cv2.VideoWriter("reflectance_output.avi", cv2.VideoWriter_fourcc(*'DIVX'), 10, (2400, 1599))

start_index = 350
stop_index = 400
pbar = tqdm(total = (stop_index - start_index), position=0, leave=True)
all_files = [o3d.io.read_point_cloud(point_cloud_files[i]) for i in range(start_index, stop_index)]

for i in range(len(all_files)):
    roi_pcd, filtered_point_cloud, pcd = lane_line_pipeline(all_files[i])
    fig = vis_pcd([pcd.paint_uniform_color((0.2,0.4,0.2)),roi_pcd], show=False, save=str(start_index+i))
    output_handle.write(cv2.imread("output/"+str(start_index+i)+"_processed.jpg"))
    pbar.update(1)

output_handle.release()

Normals Estimation and filtering the objects by using Normals. We will distuinguish the lane lines and and the barricades on the road. For do that, we need the reuse our reflectivity_thereshold and roi_thereshold.

In [ ]:
pcd = o3d.io.read_point_cloud(point_cloud_files[idx])
filtered_point_cloud=reflectivity_threshold(pcd, thresh=0.45)
roi_pcd=roi_filter(filtered_point_cloud, roi_min=(-20,-6,-2), roi_max=(20,6,0))
roi_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=10,max_nn=30))
o3d.visualization.draw_geometries([pcd.paint_uniform_color((0.8,0.8,0.8)),roi_pcd],point_show_normal=True)
# Did you see the normal lines which has not a 90°. Let's eliminate them
for i in range(100):
    if i%10==0:
        print("XYZ and normal for the point",str(i))
        print(roi_pcd.points[i])
        print(roi_pcd.normals[i])


In [ ]:
#def filtering_normals(pcd):
normals=np.asarray(roi_pcd.normals)
direction=np.array([0,0,1])
angles=np.arccos(np.dot(normals,direction))
angle_threshold=np.deg2rad(10)
mask=np.where(np.abs(angles)<angle_threshold)[0]
filtered_roi_pcd=roi_pcd.select_by_index(mask)
o3d.visualization.draw_geometries([pcd.paint_uniform_color((0.8,0.8,0.8)),filtered_roi_pcd],point_show_normal=True)

## Part II: 3D Object Detection
So let's see a few algorithms to do this.
1. Downsampling
2. 3D Segmentation
3. 3D Clustering
4. Bounding Box Fitting

### 1 — Downsampling
Creating a video took time. It's not surprising since we're processing hundreds of thousands of points! But here's a thought: do we really need all those 100k+ points? What if we could achieve the same original shape with fewer points? That's where Downsampling comes into play.

In [ ]:
pcd = o3d.io.read_point_cloud(point_cloud_files[200])
print(len(pcd.points))

In [ ]:
downpcd = pcd.voxel_down_sample(voxel_size=0.2)
print(len(downpcd.points))

### 2 — 3D Segmentation
Next, let's see how to use RANSAC (Random Simple Consensus) — an outlier detection algorithm, that can be used to fit a curve, a plane, or whatever we want. In OPEN3D, RANSAC is made this way:

**plane_model, inliers = downpcd.segment_plane(distance_threshold=distance_threshold, ransac_n=ransac_n, num_iterations=num_iterations)**

The parameters you can find in the Documentation of  [opend3d](http://www.open3d.org/docs/release/python_api/open3d.geometry.PointCloud.html?highlight=segment_plane)


In [ ]:
def ransac(pcd, distance_threshold=0.33, ransac_n=3, num_iterations=100):
    plane_model,inliers = pcd.segment_plane(distance_threshold=distance_threshold, ransac_n=ransac_n, num_iterations=num_iterations)
    inlier_cloud = pcd.select_by_index(inliers)
    outlier_cloud = pcd.select_by_index(inliers, invert=True)
    outlier_cloud.paint_uniform_color([0.5, 0.75, 0.25])
    inlier_cloud.paint_uniform_color([0.25, 0.5, 0.75])
    return inlier_cloud, outlier_cloud

inlier_cloud, outlier_cloud = ransac(downpcd, distance_threshold=0.33, ransac_n=3, num_iterations=150)
fig = vis_pcd([inlier_cloud, outlier_cloud])

Ransac with normals. THe normal estimation should improve the ransac

In [ ]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=10,max_nn=30))
inlier_cloud, outlier_cloud = ransac(downpcd, distance_threshold=0.33, ransac_n=3, num_iterations=150)
fig = vis_pcd([inlier_cloud, outlier_cloud])

### Clustering (Object Detection)
The next part is clustering. We have a set of "outliers" that contain our objects. Something natural could be to cluster these objects based on their distances. We can do that using KD-Tree algorithms, or the DBScan algorithm, built in Open3D.
Here, the function will be:

**outlier_cloud.cluster_dbscan(eps=eps, min_points=min_points)**

In [ ]:
def dbscan(outlier_cloud, eps = 1, min_points=10,roi=False):
    if roi:
        outlier_cloud=roi_filter(outlier_cloud, roi_min=(-20,-8,-2), roi_max=(20,8,0))
    labels = np.array(outlier_cloud.cluster_dbscan(eps=eps,min_points=min_points))
    max_label=labels.max()
    colors=plt.get_cmap("inferno_r")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    return outlier_cloud, labels

roi_outliers, labels = dbscan(outlier_cloud, eps=2, min_points=100,roi=False)
fig = vis_pcd([inlier_cloud, roi_outliers])

In [ ]:
print(labels)
print(len(labels))

### 3D Bounding Boxes


In [ ]:
def get_bounding_boxes(labels, outlier_cloud):
    # Extract points for each cluster
    clusters = {}
    for i, label in enumerate(labels):
        if label >= 0:
            if label not in clusters:
                clusters[label] = []
            clusters[label].append(outlier_cloud.points[i])

    clusters = {label: points for label, points in clusters.items() if len(points) <= 300}

    # Create AABB for each cluster
    aabb_boxes = []
    for points in clusters.values():
        cluster_cloud = o3d.geometry.PointCloud()
        cluster_cloud.points = o3d.utility.Vector3dVector(points)
        aabb = cluster_cloud.get_axis_aligned_bounding_box()
        aabb_boxes.append(aabb)
    return aabb_boxes

aabb_boxes = get_bounding_boxes(labels, roi_outliers)
print(len(aabb_boxes))

In [ ]:
def get_trace(obb_boxes, fig):
    width = 1.0
    height = 2.0
    depth = 3.0
    for obb in obb_boxes:
        # Get the eight corner points of the OBB
        corners = np.asarray(obb.get_box_points())

        # Extract x, y, and z coordinates of the corners
        x = corners[:, 0]
        y = corners[:, 1]
        z = corners[:, 2]
        # Create a Mesh3d trace for the oriented bounding box with opacity
        obb_trace = go.Mesh3d(
            x=x,
            y=y,
            z=z,
            i=[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 2, 6, 4, 1, 3, 7, 5],
            j=[1, 2, 3, 0, 5, 6, 7, 4, 2, 3, 7, 6, 1, 0, 4, 5, 6, 7, 3, 2, 0, 1, 5, 4],
            k=[2, 3, 0, 1, 6, 7, 4, 5, 3, 7, 6, 2, 0, 4, 5, 1, 7, 6, 2, 4, 1, 5, 0, 3],
            color='blue',
            opacity=0.3
        )

        # Add the Mesh3d trace to the figure
        fig.add_trace(obb_trace)
    return fig

In [ ]:
def pipeline(pcd):
    inlier_cloud, outlier_cloud = ransac(pcd, distance_threshold=0.33, ransac_n=3, num_iterations=100)
    roi_outliers, labels = dbscan(outlier_cloud)
    aabb_boxes = get_bounding_boxes(labels, roi_outliers)
    fig = vis_pcd([roi_outliers, inlier_cloud], show=False)
    fig = get_trace(aabb_boxes, fig)
    fig.show()

In [ ]:
pipeline(pcd)

### Video

In [ ]:
def seg_pipeline(pcd, idx):
    downpcd = pcd.voxel_down_sample(voxel_size=0.2)
    inlier_cloud, outlier_cloud = ransac(downpcd, distance_threshold=0.33, ransac_n=3, num_iterations=100)
    roi_outliers, labels = dbscan(outlier_cloud)
    aabb_boxes = get_bounding_boxes(labels, roi_outliers)
    fig = vis_pcd([downpcd, roi_outliers, inlier_cloud], show=False)
    fig = get_trace(aabb_boxes, fig)
    fig.write_image("output/"+str(idx)+"_processed_obj.jpg", scale=3)
    fig = go.Figure()

In [ ]:
import cv2
from tqdm import tqdm

output_handle = cv2.VideoWriter("objects_output.avi", cv2.VideoWriter_fourcc(*'XVID'), 10, (2400, 1599))

start_index = 180
stop_index = 230
pbar = tqdm(total = (stop_index - start_index), position=0, leave=True)
all_files = [o3d.io.read_point_cloud(point_cloud_files[i]) for i in range(start_index, stop_index)]

for i in range(len(all_files)):
    seg_pipeline(all_files[i], str(start_index+i))
    output_handle.write(cv2.imread("output/"+str(start_index+i)+"_processed_obj.jpg"))
    pbar.update(1)

output_handle.release()